In [ ]:
import torch
TORCH = torch.__version__.split('+')[0]
CUDA = 'cu' + torch.version.cuda.replace('.','')
print(TORCH, CUDA)

import pickle
import os

2.5.1 cu121


In [ ]:
%%capture
!pip install pyg_lib -f https://data.pyg.org/whl/torch-2.4.0+cu121.html
!pip install torch-scatter     -f https://data.pyg.org/whl/torch-2.4.0+cu121.html
!pip install torch-cluster -f https://data.pyg.org/whl/torch-2.4.0+cu121.html
!pip install torch-spline-conv -f https://data.pyg.org/whl/torch-2.4.0+cu121.html
!pip install torch-sparse -f https://data.pyg.org/whl/torch-2.4.0+cu121.html
!pip install torch-geometric

In [ ]:
import torch_geometric
import torch_geometric.nn as geom_nn
import torch_geometric.data as geom_data
from torch_geometric.loader import DataLoader
from torch_geometric.data import InMemoryDataset
from torch_geometric.data import Data

import numpy as np
import matplotlib.pyplot as plt
import networkx as nx

from scipy.linalg import fractional_matrix_power
from scipy.spatial.distance import pdist
from scipy.spatial.distance import squareform

import matplotlib.pyplot as plt
import matplotlib as matplotlib
import matplotlib.cm as cm

from tqdm import tqdm

import math
from numba import cuda
import numpy as np
import torch
from torch_geometric.utils import (
    to_networkx,
    from_networkx,
    to_dense_adj,
    remove_self_loops,
    to_undirected,
)

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


# Set up Functions

In [ ]:
def data_to_kNN(X,k):
  edge_index = torch_geometric.nn.knn_graph(X, k)
  edge_index = to_undirected(edge_index, num_nodes=X.shape[0])
  return edge_index

def data_to_GNN_data(X,Y,k):
  dataset = []
  N = X.shape[0]
  for i in range(N):
    data = Data(x = X[i,:,:], y = Y[i,:,:], edge_index = data_to_kNN(X[i,:,:],k))
    data.num_nodes = 20
    data.num_edges = data.edge_index.shape[1]
    dataset.append(data)
  return dataset

def data_to_GNN_data_fm(X,Y,k):
  dataset = []
  N = len(X)
  for i in tqdm(range(N)):
    data = Data(x = X[i], y = Y[i], edge_index = data_to_kNN(X[i],k))
    data.num_nodes = X[i].shape[0]
    data.num_edges = data.edge_index.shape[1]
    dataset.append(data)
  return dataset

# Preprocessing Fish Data

In [ ]:
import json

# Opening JSON file
f = open('drive/MyDrive/MeasureMaps/schooling_frames.json')

# returns JSON object as
# a dictionary
data = json.load(f)

In [ ]:
fish_ids_to_idx = dict()

idx = 0
for i in range(1,len(data)+1):
  curr_time = data[str(i)]
  fish_idx = data[str(i)]["onfish"]
  for j in range(len(fish_idx)):
    if fish_idx[j] not in fish_ids_to_idx:
      fish = dict()
      fish["id"] = idx
      fish["times"] = [i]
      fish["x"] = torch.tensor([curr_time["px"][j], curr_time["py"][j], curr_time["vx"][j], curr_time["vy"][j]]).unsqueeze(0)
      fish_ids_to_idx[fish_idx[j]] = fish
      idx += 1
    else:
      fish = fish_ids_to_idx[fish_idx[j]]
      fish["times"].append(i)
      try:
        x = torch.tensor([curr_time["px"][j], curr_time["py"][j], curr_time["vx"][j], curr_time["vy"][j]]).unsqueeze(0)
      except:
        x = torch.tensor([curr_time["px"][j], curr_time["py"][j], 0,0]).unsqueeze(0)
      fish["x"] = torch.cat((fish["x"],x), dim=0)
      fish_ids_to_idx[fish_idx[j]] = fish

In [ ]:
avg = 0
for k in fish_ids_to_idx.keys():
  avg += len(fish_ids_to_idx[k]["times"])

avg/len(fish_ids_to_idx.keys())

104.00130220444609

In [ ]:
fish_ids_to_idx[110450]["times"]

In [ ]:
def create_y_for_fish(fish):
  X = fish["x"].T
  t = fish["times"]

  x,y = create_y(X,t)
  fish["x"] = x
  fish["y"] = y
  fish["times"] = t[1:-1]
  return fish

def create_y(X,t):
  N = X.shape[0]
  T = X.shape[1]

  n = N//4

  y = torch.zeros(N,T)
  for i in range(1,T-1):
    y[:n,i] = X[2*n:3*n, i]
    y[n:2*n,i] = X[3*n:, i]
    y[2*n:,i] = (X[2*n:,i+1] - X[2*n:,i-1])/(t[i+1]-t[i-1])

  return X[:,1:-1], y[:,1:-1]

In [ ]:
for k in fish_ids_to_idx.keys():
  fish_ids_to_idx[k] = create_y_for_fish(fish_ids_to_idx[k])

In [ ]:
times_to_fish = {}
keys = fish_ids_to_idx.keys()
for t in range(1,5001):
  fish_idx = []
  for k in keys:
    if t in fish_ids_to_idx[k]["times"]:
      fish_idx.append(k)
  times_to_fish[t] = fish_idx

In [ ]:
Xs = []
Ys = []
keys = fish_ids_to_idx.keys()
for t in range(1,5001):
  fish_idxs = times_to_fish[t]
  x = torch.zeros(len(fish_idxs),4)
  y = torch.zeros(len(fish_idxs),4)
  for i,fish_id in enumerate(fish_idxs):
    fish = fish_ids_to_idx[fish_id]
    t_idx = fish["times"].index(t)
    x[i,:] = fish["x"].T[t_idx,:]
    y[i,:] = fish["y"].T[t_idx,:]
  Xs.append(x)
  Ys.append(y)

In [ ]:
torch.save((Xs, Ys), "drive/MyDrive/MeasureMaps/FishMilling.pt")

# Setting Up Models

In [ ]:
import torch
import scipy.io as sio
import torch.nn as nn

In [ ]:
# Throughout this we are going to assume that data is of the form B x N x D
# Where B is the batch size, N is the sequence length for the transformer
# this the number of data points. Finally D is the embedding dimension.
class SimpleAttention(nn.Module):
  # Initialize the parameter
  def __init__(self, hidden_dim):
    super(SimpleAttention, self).__init__()
    self.linear = nn.Linear(hidden_dim, hidden_dim)
    self.WQ = nn.Linear(hidden_dim, hidden_dim, bias = False)
    self.WK = nn.Linear(hidden_dim, hidden_dim, bias = False)
    self.WV = nn.Linear(hidden_dim, hidden_dim, bias = False)
    self.skip = nn.Linear(hidden_dim, hidden_dim)
    self.attention = nn.MultiheadAttention(hidden_dim, 1, batch_first=True)

  # Forward pass
  def forward(self, input):
    Q = self.WQ(input)
    K = self.WK(input)
    V = self.WV(input)
    output_attention,_ = self.attention(Q,K,V)
    output_linear = self.linear(output_attention.relu()).relu()
    return output_linear + self.skip(input)

class SimpleTransformer(nn.Module):
# Initialize the parameter
  def __init__(self, input_dim, hidden_dim, out_dim, num_layers):
    super(SimpleTransformer, self).__init__()
    self.embed = nn.Linear(input_dim, hidden_dim)
    self.predictor = nn.Linear(hidden_dim, out_dim)
    self.AttentionLayers = []
    for i in range(num_layers):
      self.AttentionLayers.append(SimpleAttention(hidden_dim))

    self.AttentionLayers = nn.ModuleList(self.AttentionLayers)
    self.num_layers = num_layers

  # Forward pass
  def forward(self, z):
    z = self.embed(z)
    for i in range(self.num_layers):
      z = self.AttentionLayers[i](z)
    return self.predictor(z)


class FNN(nn.Module):
  def __init__(self, input_dim, hidden_dim, out_dim, num_layers, d):
    super(FNN, self).__init__()
    self.embed = nn.Linear(input_dim, hidden_dim)
    self.predictor = nn.Linear(hidden_dim, out_dim)
    self.Layers = []
    for i in range(num_layers):
      self.Layers.append(nn.Linear(hidden_dim, hidden_dim))

    self.Layers = nn.ModuleList(self.Layers)
    self.num_layers = num_layers
    self.d = d

  # Input size B x N X 4
  def forward(self,x):
    x = torch.flatten(x, 1, 2) # B x N4
    x = self.embed(x).relu()
    for i in range(self.num_layers):
      x = self.Layers[i](x).relu()
    x = self.predictor(x)
    x = torch.unflatten(x, 1, (-1,self.d))
    return x

def kernel_basis(X, d1, d2):
  N = X.shape[0]
  d = X.shape[1]

  Phi = torch.zeros(N, (d1+2*d2)*d, device = X.device)
  for i in range(d1+2*d2):
    if i < d1:
      for j in range(d):
        Phi[:,d*i+j] = X[:,j].pow(i+1)
    elif i < d1+d2:
      for j in range(d):
        k = i-d1+1
        Phi[:,d*i+j] = (X[:,j]*k).sin()
    else:
      for j in range(d):
        k = i-d1-d2+1
        Phi[:,d*i+j] = torch.cos(X[:,j]*k)

  return Phi

class Kernel(nn.Module):
  def __init__(self, feature_dim, out_dim, d, embed=kernel_basis):
    super(Kernel, self).__init__()
    self.predictor = nn.Linear(feature_dim, out_dim, device = "cuda")
    self.embed = embed
    self.d = d

  # Input size B x N X 4
  def forward(self,x):
    x = self.predictor(x)
    x = torch.unflatten(x, 1, (-1,self.d))
    return x


In [ ]:
def get_nn(in_channels, out_channels):
  return torch.nn.Sequential(torch.nn.Linear(in_channels, out_channels), torch.nn.ReLU(),
                             torch.nn.Linear(out_channels, out_channels))

class GNN(torch.nn.Module):
  def __init__(self, node_input_dim, output_dim, num_layers, hidden_dim = 128, device = "cuda", arch = "Graph"):
    super().__init__()

    self.num_layers = num_layers
    self.layers = []
    if arch == "Transformer":
      self.layers.append(torch_geometric.nn.TransformerConv(node_input_dim, hidden_dim).to(device))
      for i in range(num_layers-1):
        self.layers.append(torch_geometric.nn.TransformerConv(hidden_dim, hidden_dim).to(device))
    elif arch == "Graph":
      self.layers.append(torch_geometric.nn.GraphConv(node_input_dim, hidden_dim).to(device))
      for i in range(num_layers-1):
        self.layers.append(torch_geometric.nn.GraphConv(hidden_dim, hidden_dim).to(device))
    elif arch == "GIN":
      self.layers.append(torch_geometric.nn.GINConv(get_nn(node_input_dim, hidden_dim)).to(device))
      for i in range(num_layers-1):
        self.layers.append(torch_geometric.nn.GINConv(get_nn(hidden_dim, hidden_dim)).to(device))

    self.layers = torch.nn.ModuleList(self.layers)
    self.lin = torch.nn.Linear(hidden_dim, output_dim).to(device)

  def forward(self, x, edge_index, batch):
    for i in range(self.num_layers):
      x = self.layers[i](x, edge_index).relu()

    z = self.lin(x)
    return z

In [ ]:
class Cylindrical_FNN(torch.nn.Module):
  def __init__(self, input_dim, hidden_dim, out_dim, num_layers):
    super(Cylindrical_FNN, self).__init__()
    self.layers = []
    self.layers.append(nn.Linear(input_dim, hidden_dim))
    for i in range(num_layers-2):
      self.layers.append(nn.Linear(hidden_dim, hidden_dim))
    self.layers.append(nn.Linear(hidden_dim, out_dim))

    self.layers = nn.ModuleList(self.layers)

  def forward(self, x):
    for i in range(len(self.layers)):
      x = self.layers[i](x).relu()
    return x

class Cylindrical(torch.nn.Module):
  def __init__(self, node_input_dim, output_dim, num_layers_psi, num_layers_phi, hidden_dim = 128, device = "cuda"):
    super().__init__()

    self.psi = Cylindrical_FNN(node_input_dim, hidden_dim, hidden_dim, num_layers_psi).to(device)
    self.phi = Cylindrical_FNN(hidden_dim+node_input_dim, hidden_dim, output_dim, num_layers_phi).to(device)

  def forward(self, x):
    z = self.psi(x)
    z_mean = z.mean(dim=1).repeat(1, x.shape[1], 1)
    z = torch.cat((z_mean,x), dim=-1)
    z = self.phi(z)

    return z

# Training

In [ ]:
X, Y = torch.load("drive/MyDrive/MeasureMaps/FishMilling.pt")

# dataset_knn = data_to_GNN_data_fm(Xtrn, Ytrn, 3)
# dataset_full = data_to_GNN_data_fm(Xtrn, Ytrn, 20)

<ipython-input-10-db67106fc7b2>:1: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  X, Y = torch.load("drive/MyDrive/MeasureMaps/FishMilling.pt")


In [ ]:
train_idx = torch.load("drive/MyDrive/MeasureMaps/FM-data-split.pt")

<ipython-input-11-497a7d560dad>:1: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  train_idx = torch.load("drive/MyDrive/MeasureMaps/FM-data-split.pt")


In [ ]:
Xtrn = [X[i] for i in train_idx]
Ytrn = [Y[i] for i in train_idx]

In [ ]:
# train_data = torch.utils.data.TensorDataset(Xtrn[train_idx,:,:], Ytrn[train_idx,:,:])

# train_loader = torch.utils.data.DataLoader(train_data, shuffle = True, batch_size=500)
# train_loader_gnn_full = DataLoader([dataset_full[i] for i in train_idx.numpy()], shuffle = True, batch_size = 500)
# train_loader_gnn_knn = DataLoader([dataset_knn[i] for i in train_idx.numpy()], shuffle = True, batch_size = 500)

TypeError: list indices must be integers or slices, not tuple

In [ ]:
from tqdm import tqdm

depths = [3,4,5]
widths = [128,256,512]
lrs = [2e-4, 1e-3, 1e-4]
T = 5
N = len(Xtrn)

epochs = 11

for t in range(T):
  for d in depths: # depth
    for h in widths: # width
      for lr in lrs:
        if os.path.exists("drive/MyDrive/MeasureMaps/Cylindrical/fm-trial-"+str(t)+"-depth-"+str(d)+"-width-"+str(h)+"-lr-"+str(lr)+"-epoch-"+str(10)+".pt"):
          print("Done with",t,d,h,lr)
          continue
        model_cylin = Cylindrical(4,4,d//2, d - d//2, h).to('cuda')

        optimizer_cylin = torch.optim.Adam(model_cylin.parameters(), lr = lr)
        scheduler_cylin = torch.optim.lr_scheduler.CosineAnnealingLR(optimizer_cylin, epochs)

        for i in tqdm(range(epochs)):
          for j in range(N):
            optimizer_cylin.zero_grad()
            X = Xtrn[j].to('cuda').unsqueeze(0)
            Y = Ytrn[j].to('cuda').unsqueeze(0)
            Y_pred = model_cylin(X)
            loss = torch.nn.functional.mse_loss(Y_pred, Y)
            loss.backward()
            optimizer_cylin.step()
          torch.save(model_cylin,"drive/MyDrive/MeasureMaps/Cylindrical/fm-trial-"+str(t)+"-depth-"+str(d)+"-width-"+str(h)+"-lr-"+str(lr)+"-epoch-"+str(i)+".pt")
          scheduler_cylin.step()

100%|██████████| 11/11 [01:30<00:00,  8.22s/it]


In [ ]:
from tqdm import tqdm

depths = [3,4,5]
widths = [128,256,512]
lrs = [2e-4, 1e-3, 1e-4]
T = 5
N = len(train_X)

epochs = 11

for t in range(T):
  for d in depths: # depth
    for h in widths: # width
      for lr in lrs:
        if os.path.exists("drive/MyDrive/MeasureMaps/Transformer/fm-trial-"+str(t)+"-depth-"+str(d)+"-width-"+str(h)+"-lr-"+str(lr)+"-epoch-"+str(10)+".pt"):
          print("Done with",t,d,h,lr)
          continue
        model_transformer = SimpleTransformer(4,h,4,d).to('cuda')

        optimizer_transformer = torch.optim.Adam(model_transformer.parameters(), lr = lr)
        scheduler_transformer = torch.optim.lr_scheduler.CosineAnnealingLR(optimizer_transformer, epochs)

        for i in tqdm(range(epochs)):
          for j in range(N):
            optimizer_transformer.zero_grad()
            X = train_X[j].to('cuda')
            Y = train_Y[j].to('cuda')
            Y_pred = model_transformer(X)
            loss = torch.nn.functional.mse_loss(Y_pred, Y)
            loss.backward()
            optimizer_transformer.step()
          # torch.save(model_transformer,"drive/MyDrive/MeasureMaps/Transformer/fm-trial-"+str(t)+"-depth-"+str(d)+"-width-"+str(h)+"-lr-"+str(lr)+"-epoch-"+str(i)+".pt")
          scheduler_transformer.step()

In [ ]:
from tqdm import tqdm

degree = [2,3,4]
frequencies = [4,5,6]
lrs = [1e-4, 2e-4, 1e-3]
T = 5

epochs = 1001

for t in range(T):
  for d1 in degree:
    for d2 in frequencies:
      for lr in lrs:
        if os.path.exists("drive/MyDrive/MeasureMaps/Kernel/trial-"+str(t)+"-depth-"+str(d)+"-width-"+str(h)+"-lr-"+str(lr)+"-epoch-"+str(1000)+".pt"):
          continue
        Xtrn_data_kernel = kernel_basis(torch.flatten(Xtrn_data,1,2),d1,d2)
        train_data_kernel = torch.utils.data.TensorDataset(Xtrn_data_kernel[train_idx,:], Ytrn_data[train_idx,:,:])
        train_loader_kernel = torch.utils.data.DataLoader(train_data_kernel, shuffle = True, batch_size=500)

        model_kernel = Kernel((d1+2*d2)*4*20,80,4).to('cuda')

        optimizer_kernel = torch.optim.Adam(model_kernel.parameters(), lr = lr)
        scheduler_kernel = torch.optim.lr_scheduler.CosineAnnealingLR(optimizer_kernel, epochs)

        for i in tqdm(range(epochs)):
          for X,Y in train_loader_kernel:
            optimizer_kernel.zero_grad()
            Y_pred = model_kernel(X)
            loss = torch.nn.functional.mse_loss(Y_pred, Y)
            loss.backward()
            optimizer_kernel.step()
          if i % 100 == 0:
            torch.save(model_kernel,"drive/MyDrive/MeasureMaps/Kernel/trial-"+str(t)+"-degree-"+str(d1)+"-frequency-"+str(d2)+"-lr-"+str(lr)+"-epoch-"+str(i)+".pt")
          scheduler_kernel.step()

In [ ]:
from tqdm import tqdm

depths = [3,4,5]
widths = [128,256,512]
lrs = [1e-4, 2e-4, 1e-3]
T = 5
N = 20

epochs = 1001

for t in range(1):
  for d in depths:
    for h in widths:
      for lr in lrs:
        if os.path.exists("drive/MyDrive/MeasureMaps/FNN/trial-"+str(t)+"-depth-"+str(d)+"-width-"+str(h)+"-lr-"+str(lr)+"-epoch-"+str(1000)+".pt"):
          continue
        model_fnn = FNN(4*N,h,4*N,d,4).to('cuda')


        optimizer_fnn = torch.optim.Adam(model_fnn.parameters(), lr = lr)
        scheduler_fnn = torch.optim.lr_scheduler.CosineAnnealingLR(optimizer_fnn, epochs)

        for i in tqdm(range(epochs)):
          for X,Y in train_loader:
            optimizer_fnn.zero_grad()
            Y_pred = model_fnn(X)
            loss = torch.nn.functional.mse_loss(Y_pred, Y)
            loss.backward()
            optimizer_fnn.step()
          if i % 100 == 0:
            torch.save(model_fnn,"drive/MyDrive/MeasureMaps/FNN/trial-"+str(t)+"-depth-"+str(d)+"-width-"+str(h)+"-lr-"+str(lr)+"-epoch-"+str(i)+".pt")
          scheduler_fnn.step()

In [ ]:
from tqdm import tqdm

depths = [3,4,5]
widths = [128,256,512]
lrs = [1e-4, 2e-4, 1e-3]
T = 5

epochs = 501

for t in range(T):
  for d in depths:
    for h in widths:
      for lr in lrs:
        # if os.path.exists("drive/MyDrive/MeasureMaps/GNN/GraphConv Full/fm-trial-"+str(t)+"-depth-"+str(d)+"-width-"+str(h)+"-lr-"+str(lr)+"-epoch-"+str(10)+".pt"):
        #   print("done with", t, d, h, lr)
        #   continue
        model_gnn = GNN(4,4,d,h).to('cuda')


        optimizer_gnn = torch.optim.Adam(model_gnn.parameters(), lr = lr)
        scheduler_gnn = torch.optim.lr_scheduler.CosineAnnealingLR(optimizer_gnn, epochs)

        for i in tqdm(range(epochs)):
          for data in train_loader_gnn_full:
            optimizer_gnn.zero_grad()
            data = data.to('cuda')
            Y_pred = model_gnn(data.x, data.edge_index, data.batch)
            loss = torch.nn.functional.mse_loss(Y_pred, data.y)
            loss.backward()
            optimizer_gnn.step()
          print(loss)
          torch.save(model_gnn,"drive/MyDrive/MeasureMaps/GNN/GraphConv Full/fm-trial-"+str(t)+"-depth-"+str(d)+"-width-"+str(h)+"-lr-"+str(lr)+"-epoch-"+str(i)+".pt")
          scheduler_gnn.step()

In [ ]:
from tqdm import tqdm

depths = [3,4,5]
widths = [128,256,512]
lrs = [1e-4, 2e-4, 1e-3]
T = 5

epochs = 11

for t in range(T):
  for d in depths:
    for h in widths:
      for lr in lrs:
        # if os.path.exists("drive/MyDrive/MeasureMaps/GNN/GraphConv KNN/fm-trial-"+str(t)+"-depth-"+str(d)+"-width-"+str(h)+"-lr-"+str(lr)+"-epoch-"+str(10)+".pt"):
        #   print("Done with",t,d,h,lr)
        #   continue
        model_gnn = GNN(4,4,d,h).to('cuda')


        optimizer_gnn = torch.optim.Adam(model_gnn.parameters(), lr = lr)
        scheduler_gnn = torch.optim.lr_scheduler.CosineAnnealingLR(optimizer_gnn, epochs)

        for i in tqdm(range(epochs)):
          for data in train_loader_gnn_knn:
            optimizer_gnn.zero_grad()
            data = data.to('cuda')
            Y_pred = model_gnn(data.x, data.edge_index, data.batch)
            loss = torch.nn.functional.mse_loss(Y_pred, data.y)
            loss.backward()
            optimizer_gnn.step()
          print(loss)
          # torch.save(model_gnn,"drive/MyDrive/MeasureMaps/GNN/GraphConv KNN/fm-trial-"+str(t)+"-depth-"+str(d)+"-width-"+str(h)+"-lr-"+str(lr)+"-epoch-"+str(i)+".pt")
          scheduler_gnn.step()

In [ ]:
from tqdm import tqdm

depths = [3,4,5]
widths = [128,256,512]
lrs = [1e-4, 2e-4, 1e-3]
T = 5

epochs = 11

for t in range(T):
  for d in depths:
    for h in widths:
      for lr in lrs:
        # if os.path.exists("drive/MyDrive/MeasureMaps/GNN/TransformerConv Full/fm-trial-"+str(t)+"-depth-"+str(d)+"-width-"+str(h)+"-lr-"+str(lr)+"-epoch-"+str(10)+".pt"):
        #   print("Done with",t,d,h,lr)
        #   continue
        model_gnn = GNN(4,4,d,h,arch="Transformer").to('cuda')


        optimizer_gnn = torch.optim.Adam(model_gnn.parameters(), lr = lr)
        scheduler_gnn = torch.optim.lr_scheduler.CosineAnnealingLR(optimizer_gnn, epochs)

        for i in tqdm(range(epochs)):
          for data in train_loader_gnn_full:
            optimizer_gnn.zero_grad()
            data = data.to('cuda')
            Y_pred = model_gnn(data.x, data.edge_index, data.batch)
            loss = torch.nn.functional.mse_loss(Y_pred, data.y)
            loss.backward()
            optimizer_gnn.step()
          print(loss)
          # torch.save(model_gnn,"drive/MyDrive/MeasureMaps/GNN/TransformerConv Full/fm-trial-"+str(t)+"-depth-"+str(d)+"-width-"+str(h)+"-lr-"+str(lr)+"-epoch-"+str(i)+".pt")
          scheduler_gnn.step()

In [ ]:
from tqdm import tqdm

depths = [3,4,5]
widths = [128,256,512]
lrs = [1e-4, 2e-4, 1e-3]
T = 5

epochs = 11

for t in range(T):
  for d in depths:
    for h in widths:
      for lr in lrs:
        if os.path.exists("drive/MyDrive/MeasureMaps/GNN/TransformerConv KNN/fm-trial-"+str(t)+"-depth-"+str(d)+"-width-"+str(h)+"-lr-"+str(lr)+"-epoch-"+str(10)+".pt"):
          print("Done with",t,d,h,lr)
          continue
        model_gnn = GNN(4,4,d,h,arch="Transformer").to('cuda')

        optimizer_gnn = torch.optim.Adam(model_gnn.parameters(), lr = lr)
        scheduler_gnn = torch.optim.lr_scheduler.CosineAnnealingLR(optimizer_gnn, epochs)

        for i in tqdm(range(epochs)):
          for data in train_loader_gnn_knn:
            optimizer_gnn.zero_grad()
            data = data.to('cuda')
            Y_pred = model_gnn(data.x, data.edge_index, data.batch)
            loss = torch.nn.functional.mse_loss(Y_pred, data.y)
            loss.backward()
            optimizer_gnn.step()
          torch.save(model_gnn,"drive/MyDrive/MeasureMaps/GNN/TransformerConv KNN/fm-trial-"+str(t)+"-depth-"+str(d)+"-width-"+str(h)+"-lr-"+str(lr)+"-epoch-"+str(i)+".pt")
          scheduler_gnn.step()

# Testing

In [ ]:
Xtrn, Ytrn = torch.load("drive/MyDrive/MeasureMaps/FishMilling.pt")
train_idx = torch.load("drive/MyDrive/MeasureMaps/FM-data-split.pt")

In [ ]:
val_idx = torch.load("drive/MyDrive/MeasureMaps/FM-val-idx.pt")
test_idx = torch.load("drive/MyDrive/MeasureMaps/FM-test-idx.pt")

In [ ]:
dataset_knn = data_to_GNN_data_fm(Xtrn, Ytrn, 3)
dataset_full = data_to_GNN_data_fm(Xtrn, Ytrn, 20)

In [ ]:
val_X = [Xtrn[i] for i in val_idx]
val_Y = [Ytrn[i] for i in val_idx]

val_loader_gnn_full = DataLoader([dataset_full[i] for i in val_idx.numpy()], shuffle = True, batch_size = 1)
val_loader_gnn_knn = DataLoader([dataset_knn[i] for i in val_idx.numpy()], shuffle = True, batch_size = 1)

test_X = [Xtrn[i] for i in test_idx]
test_Y = [Ytrn[i] for i in test_idx]

test_loader_gnn_full = DataLoader([dataset_full[i] for i in test_idx.numpy()], shuffle = True, batch_size = 1)
test_loader_gnn_knn = DataLoader([dataset_knn[i] for i in test_idx.numpy()], shuffle = True, batch_size = 1)

In [ ]:
def test_transformer_fm(model, loader_X, loader_Y):
  N = len(loader_Y)
  loss = torch.zeros(N)
  model.eval()
  for i in range(N):
    X = loader_X[i].to('cuda')
    Y = loader_Y[i].to('cuda')
    Y_pred = model(X)
    loss[i] = torch.nn.functional.mse_loss(Y_pred, Y).cpu().detach()
  return loss

In [ ]:
depths = [3,4,5]
widths = [128,256,512]
lrs = [2e-4, 1e-3, 1e-4]
T = 5

best_loss = 100000
best_d = -1
best_w = -1
best_lr = -1

for d in depths: # depth
  for h in widths: # width
    for lr in lrs:
      avg_loss = 0
      print("Testing: ",d,h,lr)
      for t in range(T):
        model = torch.load("drive/MyDrive/MeasureMaps/Transformer/fm-trial-"+str(t)+"-depth-"+str(d)+"-width-"+str(h)+"-lr-"+str(lr)+"-epoch-10.pt")
        loss = test_transformer_fm(model, val_X, val_Y)
        avg_loss += loss.mean()/T
      if avg_loss < best_loss:
        best_loss = avg_loss
        best_d = d
        best_w = h
        best_lr = lr
print(best_loss, best_d, best_w, best_lr)

In [ ]:
loss_avg = torch.zeros(5)
loss_std = torch.zeros(5)
for t in range(T):
  model = torch.load("drive/MyDrive/MeasureMaps/Transformer/fm-trial-"+str(t)+"-depth-"+str(best_d)+"-width-"+str(best_w)+"-lr-"+str(best_lr)+"-epoch-10.pt")
  loss = test_transformer_fm(model, test_X, test_Y)
  loss_avg[t] = loss.mean()
  loss_std[t] = loss.std()
print(loss_avg.mean(), loss_avg.std())

tensor(0.0222) tensor(1.8757e-05)


In [ ]:
def test_GNN_fm(model, loader):
  N = len(loader)
  loss = torch.zeros(N)
  model.eval()
  i = 0
  for data in loader:
    data = data.to('cuda')
    Y_pred = model(data.x, data.edge_index, data.batch)
    loss[i] = torch.nn.functional.mse_loss(Y_pred, data.y).cpu().detach()
    i += 1
  return loss

In [ ]:
depths = [3,4,5]
widths = [128,256,512]
lrs = [2e-4, 1e-3, 1e-4]
T = 5

best_loss = 100000
best_d = -1
best_w = -1
best_lr = -1

for d in depths: # depth
  for h in widths: # width
    for lr in lrs:
      avg_loss = 0
      print("Testing: ",d,h,lr)
      for t in range(T):
        model = torch.load("drive/MyDrive/MeasureMaps/GNN/GraphConv Full/fm-trial-"+str(t)+"-depth-"+str(d)+"-width-"+str(h)+"-lr-"+str(lr)+"-epoch-10.pt")
        loss = test_GNN_fm(model, val_loader_gnn_full)
        avg_loss += loss.mean()/T
      print(avg_loss)
      if avg_loss < best_loss:
        best_loss = avg_loss
        best_d = d
        best_w = h
        best_lr = lr
print(best_loss, best_d, best_w, best_lr)

Testing:  3 128 0.0002
tensor(24886930.)
Testing:  3 128 0.001
tensor(8389030.)
Testing:  3 128 0.0001
tensor(1.5088e+08)
Testing:  3 256 0.0002
tensor(7238146.5000)
Testing:  3 256 0.001
tensor(8501644.)
Testing:  3 256 0.0001
tensor(7180415.5000)
Testing:  3 512 0.0002
tensor(6561584.)
Testing:  3 512 0.001
tensor(5210658.)
Testing:  3 512 0.0001
tensor(4909415.)
Testing:  4 128 0.0002
tensor(1.1334e+09)
Testing:  4 128 0.001
tensor(4.5062e+08)
Testing:  4 128 0.0001
tensor(2.8523e+09)
Testing:  4 256 0.0002
tensor(4.7832e+08)
Testing:  4 256 0.001
tensor(2.3417e+08)
Testing:  4 256 0.0001
tensor(1.6139e+09)
Testing:  4 512 0.0002
tensor(3.8287e+08)
Testing:  4 512 0.001
tensor(1.9696e+08)
Testing:  4 512 0.0001
tensor(3.9414e+08)
Testing:  5 128 0.0002
tensor(5.4111e+10)
Testing:  5 128 0.001
tensor(2.2792e+10)
Testing:  5 128 0.0001
tensor(3.1073e+11)
Testing:  5 256 0.0002
tensor(2.9897e+10)
Testing:  5 256 0.001
tensor(1.2194e+10)
Testing:  5 256 0.0001
tensor(4.4744e+10)
Testing

In [ ]:
loss_avg = torch.zeros(5)
loss_std = torch.zeros(5)
for t in range(T):
  model = torch.load("drive/MyDrive/MeasureMaps/GNN/GraphConv Full/fm-trial-"+str(t)+"-depth-"+str(best_d)+"-width-"+str(best_w)+"-lr-"+str(best_lr)+"-epoch-10.pt")
  loss = test_GNN_fm(model, test_loader_gnn_full)
  loss_avg[t] = loss.mean()
  loss_std[t] = loss.std()
print(loss_avg.mean(), loss_avg.std())

tensor(0.1210) tensor(0.0207)
